In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
load_dotenv()

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('groq_api_key'),
    model_name = os.getenv('llama_model_name')
)

# response = llm.invoke("The first person to land on moon was ...")
# print(response.content)

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-i-itc/job/R-51569")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Software Engineer I , ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Lan

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPPED TEXT FROM WEBSITE:
    {page_data}
    ###  INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE): 
    """
)

chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={'page_data':page_data})

'```json\n{\n  "role": "Software Engineer I, ITC",\n  "experience": "1+ years of professional experience as a Full Stack Software Engineer or in a similar role",\n  "skills": [\n    "Front-End Skills in React or similar technologies",\n    "Server-side programming languages such as Node.js, Python or similar",\n    "NoSQL databases",\n    "API Development: Hands on exposure to designing and consuming RESTful APIs and web services",\n    "Cloud Technologies: Familiarity with cloud platforms preferably AWS, including experience with serverless architectures and cloud-native development",\n    "Version Control: Proficient with Git, GitHub, or similar version control tools",\n    "Testing Frameworks: Experience with unit testing, integration testing, and test-driven development (TDD)"\n  ],\n  "description": "Design & Develop: Build, test, and maintain scalable web applications and services. Work on both front-end and back-end development to deliver end-to-end functionality. Collaborate: W

In [5]:
print(response.content)

```json
{
  "role": "Software Engineer I, ITC",
  "experience": "1+ years of professional experience as a Full Stack Software Engineer or in a similar role",
  "skills": [
    "Front-End Skills in React or similar technologies",
    "Server-side programming languages such as Node.js, Python or similar",
    "NoSQL databases",
    "API Development: Hands on exposure to designing and consuming RESTful APIs and web services",
    "Cloud Technologies: Familiarity with cloud platforms preferably AWS, including experience with serverless architectures and cloud-native development",
    "Version Control: Proficient with Git, GitHub, or similar version control tools",
    "Testing Frameworks: Experience with unit testing, integration testing, and test-driven development (TDD)"
  ],
  "description": "Design & Develop: Build, test, and maintain scalable web applications and services. Work on both front-end and back-end development to deliver end-to-end functionality. Collaborate: Work closely wi

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(response.content)
json_res

{'role': 'Software Engineer I, ITC',
 'experience': '1+ years of professional experience as a Full Stack Software Engineer or in a similar role',
 'skills': ['Front-End Skills in React or similar technologies',
  'Server-side programming languages such as Node.js, Python or similar',
  'NoSQL databases',
  'API Development: Hands on exposure to designing and consuming RESTful APIs and web services',
  'Cloud Technologies: Familiarity with cloud platforms preferably AWS, including experience with serverless architectures and cloud-native development',
  'Version Control: Proficient with Git, GitHub, or similar version control tools',
  'Testing Frameworks: Experience with unit testing, integration testing, and test-driven development (TDD)'],
 'description': 'Design & Develop: Build, test, and maintain scalable web applications and services. Work on both front-end and back-end development to deliver end-to-end functionality. Collaborate: Work closely with product managers, designers, an

In [8]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')

df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents = row['Techstack'],
            metadatas = {"links":row['Links']},
            ids = [str(uuid.uuid4())]
        )

In [10]:
job = json_res

In [11]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
email_response = chain_email.invoke({'job_description':str(job),
                                     'link_list':links})

In [14]:
print(email_response.content)

Subject: Expert Software Engineering Services for Your Business Needs

Dear Hiring Manager,

I came across the job description for a Software Engineer I, ITC, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can fulfill your requirements. With our expertise in designing and developing scalable web applications, we can help you build, test, and maintain high-quality software solutions.

Our team of skilled engineers has hands-on experience with front-end technologies like React, as well as server-side programming languages such as Node.js and Python. We're well-versed in NoSQL databases, API development, and cloud technologies, including AWS and serverless architectures. Our proficiency in version control tools like Git and GitHub, along with testing frameworks, ensures that our solutions are reliable and efficient.

At AtliQ, we've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heighte